In [1]:
!pip install sentence-transformers scikit-learn

Setting up the KB (knowledge base) and defining it

In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# The Knowledge Base (KB) from Step 1
KB_CHUNKS = [
    """Vegan Cookie Baking: Core Ingredient Swaps
Successful vegan cookie baking primarily involves replacing animal products with plant-based alternatives while maintaining the desired texture and flavor. The most common swaps involve butter and eggs. For butter, coconut oil, vegan butter sticks, or shortening are excellent 1:1 substitutes, contributing richness and helping with spread. When replacing eggs, which act as a binder and leavening agent, popular options include flax eggs (1 tablespoon of ground flaxseed mixed with 3 tablespoons of water, rested for 5 minutes), commercial egg replacers, or mashed ripe banana (adds moisture and slight sweetness). Choosing the right combination depends on the cookie's final desired desired texture, such as chewier vs. crispier.""",
    
    """Achieving the Perfect Vegan Cookie Texture
The key to texture often lies in the fat-to-flour ratio and baking time. For a **chewy** cookie, use a higher ratio of moist ingredients (like brown sugar over white sugar) and slightly underbake them. Chilling the dough before baking is crucial as it solidifies the fat, preventing the cookies from spreading too quickly, which results in a thicker, chewier final product. For **crispy** cookies, use less moisture, more white sugar, and slightly overbake until the edges are deep golden brown. The choice of fat also impacts texture; shortening tends to produce a more tender crumb than coconut oil.""",

    """Flavor Enhancements and Common Pitfalls
To enhance the flavor of vegan cookies, consider using vanilla bean paste instead of extract for a richer taste, or adding a pinch of salt to balance the sweetness. A common pitfall is the density that can result from overmixing the dough, especially after adding the flour. Overmixing develops the gluten, leading to tough, cake-like cookies instead of tender ones. Another issue is using melted vegan butter when the recipe calls for softened; this significantly changes the dough's consistency and will cause excessive spreading. Always follow the recipe's instructions regarding the temperature of the fat."""
]

# Create a DataFrame for easy indexing
kb_df = pd.DataFrame({'chunk_text': KB_CHUNKS})

Loading the model and generating Embeddings

In [3]:
# Load a lightweight pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded.")

# Generate embeddings for all KB chunks
KB_EMBEDDINGS = model.encode(kb_df['chunk_text'].tolist(), convert_to_numpy=True)
kb_df['embedding'] = list(KB_EMBEDDINGS)

print(f"Embeddings generated for {len(KB_EMBEDDINGS)} chunks.")
print(f"Embedding dimension: {KB_EMBEDDINGS.shape[1]}")

# You can display the dataframe to confirm the embeddings were added
# display(kb_df.head())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\olive\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\olive\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded.
Embeddings generated for 3 chunks.
Embedding dimension: 384


Definings the functions for retrieval and prompt construction. Using generate_answer function as a simulated LLM which uses logic to provide expected answers for test cases.

def retrieve_context takes a query, embeds it, calculates similarity, and returns the top relevant chunks.

In [4]:
def retrieve_context(user_query, top_k=2):
    """
    Generates query embedding and retrieves top_k most similar chunks 
    from the indexed KB_EMBEDDINGS.
    """
    # 1. Generate Query Embedding
    query_embedding = model.encode(user_query, convert_to_numpy=True)

    # 2. Calculate Cosine Similarity
    # Reshape query for similarity calculation (1, D) vs (N, D)
    similarities = cosine_similarity(query_embedding.reshape(1, -1), KB_EMBEDDINGS)[0]

    # 3. Get Top-K Indices
    # argsort returns indices that would sort the array; [::-1] reverses it to get highest first
    top_indices = np.argsort(similarities)[::-1][:top_k]

    # 4. Retrieve Context Chunks
    retrieved_chunks = kb_df.iloc[top_indices]['chunk_text'].tolist()
    
    # Return the combined context as a single string
    return "\n---\n".join(retrieved_chunks)

Using the generate_answer function to construct the final prompt and simulate LLM response behavior.

In [5]:
def generate_answer(query, context):
    """
    Constructs the augmented prompt and simulates getting an answer from the LLM.
    
    In a real scenario, you would send PROMPT_TEMPLATE to an API or a loaded LLM model.
    """
    # 1. Construct the Augmented Prompt
    PROMPT_TEMPLATE = f"""
You are a vegan cookie baking expert. Use ONLY the provided CONTEXT to answer the QUESTION.
If the CONTEXT does not contain the answer, state clearly that you cannot answer based on the provided information.

CONTEXT:
---
{context}
---

QUESTION: {query}

ANSWER:
"""
    print("--- Augmented Prompt Sent to LLM (for reference) ---")
    print(PROMPT_TEMPLATE)
    print("--------------------------------------------------")

    # --- SIMULATED LLM RESPONSE LOGIC ---
    # This simulation mimics how a well-trained LLM would respond based on the context.
    
    if "history" in query.lower() and "history" not in context.lower():
        return "I cannot answer the question about the history of the chocolate chip cookie based *only* on the provided information, as the context focuses on vegan ingredient swaps, texture, and common baking pitfalls."
    
    elif "chewy" in query.lower() and "ingredients" in query.lower():
        # Synthesis case (combining info from Swaps and Texture chunks)
        return "To make a thick, chewy vegan cookie, you should use a fat substitute like vegan butter or coconut oil, and an egg replacer like a flax egg. For texture, the key is to use a higher ratio of moist ingredients (like brown sugar over white sugar), slightly underbake the cookies, and **chilling the dough** before baking to prevent spreading."
        
    elif "substitutions" in query.lower() or "swaps" in query.lower():
        # Factual case (answered by Chunk 0)
        return "The most common swaps are for butter and eggs. Butter can be substituted 1:1 with coconut oil, vegan butter sticks, or shortening. Eggs can be replaced with flax eggs (1 tablespoon of ground flaxseed + 3 tablespoons of water), commercial egg replacers, or mashed ripe banana."
    
    elif "overmixing" in query.lower():
        # Factual case (answered by Chunk 2)
        return "Overmixing the dough, especially after adding the flour, is a common pitfall that can lead to density. It develops the gluten, resulting in tough, cake-like cookies instead of tender ones."
        
    else:
        return "The simulated LLM was unable to generate a precise answer based on the provided query and context."

Testing by running 3 test cases using the functions from earlier

Test 1 - Factuality:

In [6]:
query_1 = "What are the common substitutions for eggs and butter in vegan baking?"
context_1 = retrieve_context(query_1, top_k=1) # Only need the most relevant for this one
answer_1 = generate_answer(query_1, context_1)

print(f"\nQUERY: {query_1}")
print(f"FINAL ANSWER:\n{answer_1}")

--- Augmented Prompt Sent to LLM (for reference) ---

You are a vegan cookie baking expert. Use ONLY the provided CONTEXT to answer the QUESTION.
If the CONTEXT does not contain the answer, state clearly that you cannot answer based on the provided information.

CONTEXT:
---
Vegan Cookie Baking: Core Ingredient Swaps
Successful vegan cookie baking primarily involves replacing animal products with plant-based alternatives while maintaining the desired texture and flavor. The most common swaps involve butter and eggs. For butter, coconut oil, vegan butter sticks, or shortening are excellent 1:1 substitutes, contributing richness and helping with spread. When replacing eggs, which act as a binder and leavening agent, popular options include flax eggs (1 tablespoon of ground flaxseed mixed with 3 tablespoons of water, rested for 5 minutes), commercial egg replacers, or mashed ripe banana (adds moisture and slight sweetness). Choosing the right combination depends on the cookie's final desi

It generated that suprisingly fast! The final answer is correct

Test 2 - Foil/General, that is asking a question which is not in the KB but in which the LLM should rely on general knowledge or state that it cannot answer.

In [7]:
query_2 = "What is the history of the chocolate chip cookie?"
context_2 = retrieve_context(query_2, top_k=1) 
answer_2 = generate_answer(query_2, context_2)

print(f"\nQUERY: {query_2}")
print(f"FINAL ANSWER:\n{answer_2}")

--- Augmented Prompt Sent to LLM (for reference) ---

You are a vegan cookie baking expert. Use ONLY the provided CONTEXT to answer the QUESTION.
If the CONTEXT does not contain the answer, state clearly that you cannot answer based on the provided information.

CONTEXT:
---
Achieving the Perfect Vegan Cookie Texture
The key to texture often lies in the fat-to-flour ratio and baking time. For a **chewy** cookie, use a higher ratio of moist ingredients (like brown sugar over white sugar) and slightly underbake them. Chilling the dough before baking is crucial as it solidifies the fat, preventing the cookies from spreading too quickly, which results in a thicker, chewier final product. For **crispy** cookies, use less moisture, more white sugar, and slightly overbake until the edges are deep golden brown. The choice of fat also impacts texture; shortening tends to produce a more tender crumb than coconut oil.
---

QUESTION: What is the history of the chocolate chip cookie?

ANSWER:

----

Test passes

Test 3 - Synthesis - A question which requires the LLM to combine information from multiple retrieved chunks to anser correctly.

In [8]:
query_3 = "I want to make a thick, chewy vegan cookie, what ingredients and techniques should I use?"
context_3 = retrieve_context(query_3, top_k=2) # Need two chunks for synthesis
answer_3 = generate_answer(query_3, context_3)

print(f"\nQUERY: {query_3}")
print(f"FINAL ANSWER:\n{answer_3}")

--- Augmented Prompt Sent to LLM (for reference) ---

You are a vegan cookie baking expert. Use ONLY the provided CONTEXT to answer the QUESTION.
If the CONTEXT does not contain the answer, state clearly that you cannot answer based on the provided information.

CONTEXT:
---
Achieving the Perfect Vegan Cookie Texture
The key to texture often lies in the fat-to-flour ratio and baking time. For a **chewy** cookie, use a higher ratio of moist ingredients (like brown sugar over white sugar) and slightly underbake them. Chilling the dough before baking is crucial as it solidifies the fat, preventing the cookies from spreading too quickly, which results in a thicker, chewier final product. For **crispy** cookies, use less moisture, more white sugar, and slightly overbake until the edges are deep golden brown. The choice of fat also impacts texture; shortening tends to produce a more tender crumb than coconut oil.
---
Flavor Enhancements and Common Pitfalls
To enhance the flavor of vegan cook

This is a great answer, some of you should take note!